# Expand Dataset with Additional Countries
## Data collection for expanded training dataset

Goal: collect world bank data for additional countries to expand from 3 countries (42 rows) to ~15-17 countries (~200+ rows)

Countries to add:
- High-risk (5): Angola, Venezuela, Zimbabwe, Iraq, Ukraine
- Medium-risk (4): Brazil, South Africa, India, Philippines  
- Low-risk (7): Norway, Denmark, Singapore, Australia, New Zealand, Switzerland, Germany

Timeframe: 2010-2023 (same as baseline)


In [ ]:
import wbdata
import pandas as pd
import datetime
import os

# set working directory to project root
# if running from notebooks/, go up one level; otherwise assume already at root
current_dir = os.getcwd()
if current_dir.endswith('notebooks'):
    os.chdir('..')
elif 'notebooks' in current_dir:
    # if notebooks is in the path, go to project root
    project_root = current_dir.split('notebooks')[0].rstrip('/')
    if os.path.exists(project_root):
        os.chdir(project_root)

# verify we're in the right place
print(f"Working directory: {os.getcwd()}")
print(f"Data file exists: {os.path.exists('data/raw/corruption_data_baseline.csv')}")


## Setting up country lists by risk category


In [ ]:
# original baseline countries (already have data)
baseline_countries = {
    'CAN': 'Canada',  # low-risk control
    'MYS': 'Malaysia',  # high-risk (1MDB)
    'MOZ': 'Mozambique'  # high-risk (hidden debt)
}

# high-risk countries: known corruption scandals, weak governance
high_risk_countries = {
    'AGO': 'Angola',  # oil revenue corruption
    'VEN': 'Venezuela',  # PDVSA corruption, collapsing governance
    'ZWE': 'Zimbabwe',  # infrastructure project corruption
    'IRQ': 'Iraq',  # reconstruction fund corruption
    'UKR': 'Ukraine'  # pre-2014 development fund issues
}

# medium-risk countries: mixed governance scores, isolated incidents
medium_risk_countries = {
    'BRA': 'Brazil',  # lava jato but stronger institutions
    'ZAF': 'South Africa',  # state capture but decent baseline
    'IND': 'India',  # mixed governance, large economy
    'PHL': 'Philippines'  # variable governance scores
}

# low-risk countries: stable, high-governance
low_risk_countries = {
    'NOR': 'Norway',  # consistently top scores
    'DNK': 'Denmark',  # strong anti-corruption
    'SGP': 'Singapore',  # high effectiveness and rule of law
    'AUS': 'Australia',  # stable governance
    'NZL': 'New Zealand',  # clean governance record
    'CHE': 'Switzerland',  # strong institutions
    'DEU': 'Germany'  # solid governance throughout period
}

# combine all new countries (excluding baseline)
all_new_countries = {**high_risk_countries, **medium_risk_countries, **low_risk_countries}

print(f"baseline countries: {len(baseline_countries)}")
print(f"high-risk countries: {len(high_risk_countries)}")
print(f"medium-risk countries: {len(medium_risk_countries)}")
print(f"low-risk countries: {len(low_risk_countries)}")
print(f"total new countries: {len(all_new_countries)}")
print(f"\nnew country codes: {list(all_new_countries.keys())}")


## Defining indicators (same as baseline)


In [ ]:
# defining all indicators we want to pull
# same as baseline notebook - split into governance indicators (main focus) and economic indicators (context)

indicators = {
    # governance indicators - these match table 1 from morgan's case study
    'VA.EST': 'Voice_Accountability',
    'PV.EST': 'Political_Stability',
    'GE.EST': 'Government_Effectiveness',
    'RQ.EST': 'Regulatory_Quality',
    'RL.EST': 'Rule_of_Law',
    'CC.EST': 'Control_of_Corruption',
    
    # economic indicators - useful for detecting financial patterns
    'DT.DOD.DECT.GN.ZS': 'External_Debt_perc_GNI',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth_annual_perc',
    'GC.XPN.TOTL.GD.ZS': 'Govt_Expenditure_perc_GDP',
    'BX.KLT.DINV.WD.GD.ZS': 'FDI_Inflows_perc_GDP',
    'SI.POV.DDAY': 'Poverty_Headcount_Ratio'
}


## Loading baseline data


In [ ]:
# load the baseline data
df_baseline = pd.read_csv('data/raw/corruption_data_baseline.csv')

print(f"baseline dataset shape: {df_baseline.shape[0]} rows, {df_baseline.shape[1]} columns")
print(f"baseline countries: {df_baseline['Country'].unique()}")
print(f"baseline years: {df_baseline['Year'].min()} to {df_baseline['Year'].max()}")


## Fetching data from world bank api for new countries


In [ ]:
# date range for historical data
data_range = (datetime.datetime(2010, 1, 1), datetime.datetime(2024, 1, 1))

# get country codes for new countries
new_country_codes = list(all_new_countries.keys())

# fetching data from api
# get_dataframe pulls all indicators for specified countries and dates
df_new = wbdata.get_dataframe(indicators, 
                              country=new_country_codes, 
                              date=data_range,
                              parse_dates=False)  # keep dates as year strings


## Cleaning and formatting the dataframe


In [ ]:
# reset index so country and date become regular columns
df_new = df_new.reset_index()
df_new = df_new.rename(columns={'date': 'Year', 'country': 'Country'})

# reorder columns for readability
column_order = ['Country', 'Year'] + list(indicators.values())
existing_columns = [col for col in column_order if col in df_new.columns]
df_new = df_new[existing_columns]

# sort by country then year
df_new = df_new.sort_values(by=['Country', 'Year']).reset_index(drop=True)


## Inspecting the data


In [ ]:
print(f"shape: {df_new.shape[0]} rows, {df_new.shape[1]} columns")
print(f"years covered: {df_new['Year'].min()} to {df_new['Year'].max()}")
print(f"\ncountries in new data:")
print(df_new['Country'].unique())

# checking for missing values
print("\nmissing values per column:")
print(df_new.isnull().sum())
print(f"\nmissing data percentage:")
print(round(df_new.isnull().sum() / len(df_new) * 100, 2))


## Combining with baseline data


In [ ]:
# combine baseline and new data
# ensure same columns
common_columns = list(set(df_baseline.columns) & set(df_new.columns))
df_baseline_subset = df_baseline[common_columns]
df_new_subset = df_new[common_columns]

# combine
df_expanded = pd.concat([df_baseline_subset, df_new_subset], ignore_index=True)

# sort by country then year
df_expanded = df_expanded.sort_values(by=['Country', 'Year']).reset_index(drop=True)

print(f"combined dataset shape: {df_expanded.shape[0]} rows, {df_expanded.shape[1]} columns")
print(f"total countries: {df_expanded['Country'].nunique()}")
print(f"\ncountries in expanded dataset:")
print(sorted(df_expanded['Country'].unique()))


## Saving to data/raw/


In [ ]:
# create directory if it doesn't exist
os.makedirs('data/raw', exist_ok=True)

# save the expanded dataset
output_path = 'data/raw/corruption_data_expanded.csv'
df_expanded.to_csv(output_path, index=False)

print(f"saved to: {output_path}")


## Quick analysis of governance indicators

checking how new countries compare across the six governance indicators


In [ ]:
# filtering to just governance indicators
governance_cols = ['Country', 'Year', 'Voice_Accountability', 'Political_Stability', 
                   'Government_Effectiveness', 'Regulatory_Quality', 'Rule_of_Law', 
                   'Control_of_Corruption']

gov_df = df_expanded[governance_cols]

# calculate average scores by country
print("average governance scores by country (2010-2023):")
print(gov_df.groupby('Country')[governance_cols[2:]].mean().round(2))
